# Migating Generative AI with Avanced Prompt Engineering
Copyright 2024 Denis Rothman, MIT License

The goal of this notebook is to introduce some Generative AI mitigating tools for educational purposes. Each tools is presented separately to provide ideas that can be expanded as deemed fit for a project or simply consulted.




#Prompt Engineering

Advanced prompt engineering is not limited to entering a good prompt or question. This notebook will show you how to implement a transformer efficiently with advanced prompt engineering:<br>

Moderation, quality control<br>

1.& 5. Input/output Moderation   
2.Building the Knowledge base(KB)<br>
3.Parsing the user's prompt and accessing the KB<br>
4.Generating ChatGPT content with a dialog function and Token management<br>






### Installing OpenAI

In [ ]:
#Importing openai
try:
  import openai
  from openai import OpenAI
except:
  !pip install openai
  import openai
  from openai import OpenAI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.1/212.1 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


###Your API Key

In [ ]:
#2.API Key
#Store you key in a file and read it(you can type it directly in the notebook but it will be visible for somebody next to you)
from google.colab import drive
drive.mount('/content/drive')
f = open("drive/MyDrive/files/api_key.txt", "r")
API_KEY=f.readline()
f.close()

#The OpenAI Key
import os
os.environ['OPENAI_API_KEY'] =API_KEY
openai.api_key = os.getenv("OPENAI_API_KEY")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#1.& 5.Input and output moderation with transformers and rule-bases

OpenAI provides general purpose [tools](https://platform.openai.com/docs/guides/moderation) to control the input and output flow of their models.
However, Pre-processing prompts and post-processing completions with a customized system can
also be useful in a specific corporate environment(security, confidentiality).


In [ ]:
text="This is a good sentence but a distasteful topic."

In [ ]:
import requests

# URL for the OpenAI moderation API
url = "https://api.openai.com/v1/moderations"

# Headers including the Authorization and Content-Type
headers = {
    "Authorization": f"Bearer {openai.api_key}",
    "Content-Type": "application/json"
}

# Data payload
data = {
    "input": text
}

# POST request
response = requests.post(url, json=data, headers=headers)

# Print the response (or handle it as needed)
print(response.json())

{'id': 'modr-8grfpWLTc1PyKfR1vXU2QCmlKH2SQ', 'model': 'text-moderation-006', 'results': [{'flagged': False, 'categories': {'sexual': False, 'hate': False, 'harassment': False, 'self-harm': False, 'sexual/minors': False, 'hate/threatening': False, 'violence/graphic': False, 'self-harm/intent': False, 'self-harm/instructions': False, 'harassment/threatening': False, 'violence': False}, 'category_scores': {'sexual': 0.00015055951371323317, 'hate': 2.1322126997347368e-07, 'harassment': 5.828445864608511e-06, 'self-harm': 1.9536396944630496e-09, 'sexual/minors': 1.25559722619073e-08, 'hate/threatening': 1.680358907574231e-10, 'violence/graphic': 3.0396016370559664e-08, 'self-harm/intent': 2.2786435238675296e-10, 'self-harm/instructions': 2.5347517487728233e-11, 'harassment/threatening': 6.804071195354311e-10, 'violence': 1.5384037396870553e-05}}]}


In [ ]:
def rule_base(text):
    words = ['bad', 'distasteful', 'evil', 'unproductive', 'null']

    for word in words:
        if word in text:
            print("Input flagged: True")
            return
    print("Input flagged: False")



In [ ]:
# Test the function
rule_base("This is a distasteful example")
rule_base("This is a good example")

Input flagged: True
Input flagged: False


Displaying the details of the response(JSON object)

In [ ]:
response_dict = response.json()

# Access the 'flagged' value
flagged = response_dict["results"][0]["flagged"]

# Print the 'flagged' value
print("Flagged:", flagged)

Flagged: False


In [ ]:
# Access the 'categories' value
categories = response_dict["results"][0]["categories"]

# Print the 'categories' value
print("Categories:", categories)

Categories: {'sexual': False, 'hate': False, 'harassment': False, 'self-harm': False, 'sexual/minors': False, 'hate/threatening': False, 'violence/graphic': False, 'self-harm/intent': False, 'self-harm/instructions': False, 'harassment/threatening': False, 'violence': False}


In [ ]:
# Access the 'category_scores' value
category_scores = response_dict["results"][0]["category_scores"]

# Print the 'category_scores' value
print("Categories:", category_scores)

Categories: {'sexual': 0.00015055951371323317, 'hate': 2.1322126997347368e-07, 'harassment': 5.828445864608511e-06, 'self-harm': 1.9536396944630496e-09, 'sexual/minors': 1.25559722619073e-08, 'hate/threatening': 1.680358907574231e-10, 'violence/graphic': 3.0396016370559664e-08, 'self-harm/intent': 2.2786435238675296e-10, 'self-harm/instructions': 2.5347517487728233e-11, 'harassment/threatening': 6.804071195354311e-10, 'violence': 1.5384037396870553e-05}


Flagged as sensitive "True" or "False"

# 2.Building a knowledge base for ChatGPT, GPT-4


In [ ]:
assert1={'role': 'assistant', 'content': 'Opening hours of Snap-LM Consulting :Monday through Friday 9am to 5pm. Services :expert systems, rule-based systems, machine learning, deep learning, transformer models.'}
assert2={'role': 'assistant', 'content': 'Services :expert systems, rule-based systems, machine learning, deep learning, transformer models.'}
assert3={'role': 'assistant', 'content': 'Services :Fine-tuning OpenAI GPT-3 models, designing datasets, designing knowledge bases.'}
assertn={'role': 'assistant', 'content': 'Services:advanced prompt engineering using a knowledge base and SEO keyword methods.'}

#Using the knowledge base as a dataset:
kbt = []
kbt.append(assert1)
kbt.append(assert2)
kbt.append(assert3)
kbt.append(assertn)

In [ ]:
#displaying the KB as a DataFrame(DF) Clic on the magic Google Colaboratory Wand to obtain a cool display
import pandas as pd
df=pd.DataFrame(kbt)
df

role                                            content
0  assistant  Opening hours of Snap-LM Consulting :Monday th...
1  assistant  Services :expert systems, rule-based systems, ...
2  assistant  Services :Fine-tuning OpenAI GPT-3 models, des...
3  assistant  Services:advanced prompt engineering using a k...

Let's add some metadata keywords for each record of the knowledge base

In [ ]:
assertkw1="open"
assertkw2="expert"
assertkw3="services"
assertkwn="prompt"

In [ ]:
#create a kb keywords as list
kbkw=[assertkw1,assertkw2,assertkw3,assertkwn]
#displaying the KB as a DataFrame(DF) Clic on the magic Google Colaboratory Wand to obtain a cool display
dfk=pd.DataFrame(kbkw)
dfk

0
0      open
1    expert
2  services
3    prompt

#3.Parsing the user requests and accessing the KB







In [ ]:
user_requests=[]
user_requests.append({'role': 'user', 'content': 'At what time does Snap-LM Consulting open on Monday?'})
user_requests.append({'role': 'user', 'content': 'At what time does Snap-LM Consulting open on Saturday?'})
user_requests.append({'role': 'user', 'content': 'Can you create an AI-driven expert system?'})
user_requests.append({'role': 'user', 'content': 'What services does Snap-LM Consulting offer?'})

In [ ]:
n=len(user_requests)

In [ ]:
# This is an example. You can customize this as you wish for your project
def parse_user(uprompt,kbkw,kbt):
  i=0
  j=0
  for kw in kbkw:
    #print(i,kw)
    rq=str(uprompt)
    k=str(kw)
    fi=rq.find(k)
    if fi>-1:
      print(kw,rq,kbt[i])
      j=i
    i+=1
  return kbt[j]


#4.Generating ChatGPT content with a dialog function

Generating content with a function that fits the user's request using OpenAI ChatGPT through the gpt-3.5 turbo model.

This function will receive an automated prompt.


In [ ]:
#convmodel="gpt-3.5-turbo"
convmodel="gpt-4"
def dialog(iprompt):
    client = OpenAI()
    response = client.chat.completions.create(
        model=convmodel,
        messages=iprompt
    )
    return response

## Implementing ChatGPT with a simplified search engine approach

The following cell contains 6 steps: <br>

Step 1: iterating through the user's requests   <br>
Step 2: the application goes through a the user's request and searches for keywords with a search-engine-like technique<br>
Step 3: the application creates a prompt, with a system message, the knowledge base record found and the initial user's request <br>
Step 4: The prompt is sent to the ChatGPT dialog function<br>
Step 5: storing the response in a list<br>
Step 6; displaying the KB as a DataFrame(DF);click on the magic Google Colaboratory Wand to obtain a cool display


5. Post-Processsing Moderation

See section 1.Moderation with transformers dans rule-bases

In [ ]:
responses=[]      #creating a list to store the dialog

#going through the user's requests in a batch for the ChatGPT simulation
for i in range(n):
  # Step 1: iterating through the user's requests
  user_request_num=i

  #Step 2: the application goes through a the user's request and searches for keywords with a search-engine-like technique
  #        to find a record in the knowledge base
  kb_record=parse_user(user_requests[user_request_num],kbkw,kbt)

  #Step 3: the application creates a prompt, with a system message, the knowledge base record found and the initial user's request
  iprompt = []
  iprompt.append({"role": "system", "content": "You are an assistant for Snap-LM Consulting."})
  iprompt.append(kb_record)
  iprompt.append(user_requests[user_request_num])

  #print(iprompt)

  #Step 4: The prompt is sent to the ChatGPT dialog function
  response = dialog(iprompt)

  #Step 5: storing the response in a list
  ex=response.choices[0].message.content
  rt = "Total Tokens:" + str(response.usage.total_tokens)
  responses.append([user_requests[user_request_num],ex,rt])

#Step 6; displaying the KB as a DataFrame(DF);click on the magic Google Colaboratory Wand to obtain a cool display
pd.DataFrame(responses, columns=['request', 'response', 'tokens'])

open {'role': 'user', 'content': 'At what time does Snap-LM Consulting open on Monday?'} {'role': 'assistant', 'content': 'Opening hours of Snap-LM Consulting :Monday through Friday 9am to 5pm. Services :expert systems, rule-based systems, machine learning, deep learning, transformer models.'}
open {'role': 'user', 'content': 'At what time does Snap-LM Consulting open on Saturday?'} {'role': 'assistant', 'content': 'Opening hours of Snap-LM Consulting :Monday through Friday 9am to 5pm. Services :expert systems, rule-based systems, machine learning, deep learning, transformer models.'}
expert {'role': 'user', 'content': 'Can you create an AI-driven expert system?'} {'role': 'assistant', 'content': 'Services :expert systems, rule-based systems, machine learning, deep learning, transformer models.'}
services {'role': 'user', 'content': 'What services does Snap-LM Consulting offer?'} {'role': 'assistant', 'content': 'Services :Fine-tuning OpenAI GPT-3 models, designing datasets, designing 

request  \
0  {'role': 'user', 'content': 'At what time does...   
1  {'role': 'user', 'content': 'At what time does...   
2  {'role': 'user', 'content': 'Can you create an...   
3  {'role': 'user', 'content': 'What services doe...   

                                            response            tokens  
0         Snap-LM Consulting opens at 9am on Monday.   Total Tokens:86  
1  I'm sorry for any confusion, but Snap-LM Consu...  Total Tokens:110  
2  As an AI, I don't physically create systems bu...  Total Tokens:341  
3  Snap-LM Consulting offers a wide range of serv...  Total Tokens:248